In [1]:
import time
from datetime import timedelta

import html
import re

import os
import ntpath

import subprocess as sp

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Enter full path to xml file

In [2]:
common_path = '../my_codesnippet_analysis'
xml_path = 'pmdxmlanalysisreports_xml/pmd*.xml'
xml_file = '{}/{}'.format(common_path, xml_path)

## Read xml file in block sizes

In [3]:
## Read xml file in block sizes
bg = bd.read_text(xml_file, blocksize='10MB', linedelimiter='</violation>\n')

In [4]:
bg

dask.bag<bag-from-delayed, npartitions=1154>

In [5]:
bg.take(1)

('<?xml version="1.0" encoding="UTF-8"?>\n<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.39.0" timestamp="2022-02-12T13:11:43.504">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10000096_9914015_73_1.java">\n<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',)

In [6]:
bg.take(5)

('<?xml version="1.0" encoding="UTF-8"?>\n<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.39.0" timestamp="2022-02-12T13:11:43.504">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10000096_9914015_73_1.java">\n<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="C

## Replace the xml header with empty string '<xml ...>' => ''

In [7]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*\?xml.*\?>\n', '', line)) 

In [8]:
bg.take(5)

('<pmd xmlns="http://pmd.sourceforge.net/report/2.0.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://pmd.sourceforge.net/report/2.0.0 http://pmd.sourceforge.net/report_2_0_0.xsd" version="6.39.0" timestamp="2022-02-12T13:11:43.504">\n<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10000096_9914015_73_1.java">\n<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="ClassNamingConventions" ruleset="Code Sty

## Replace the pmd header with empty string '<pmd  ...>' => ''

In [9]:
## Replace the xml header with empty string '<xml ...>' => ''
bg = bg.map(lambda line: re.sub(r'\s*<\s*pmd.*>\n', '', line)) 

In [10]:
bg.take(5)

('<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10000096_9914015_73_1.java">\n<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="ClassNamingConventions" ruleset="Code Style" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_codestyle.html#classnamingconventions" priority="1">\nThe class name \'Code_10000096_9914015_73_1\' doesn\'t match \'[A-Z][a-zA-Z0-9]*\'\n</violation>\n',
 '<viola

### Drop wherever the Errors tage occurs

In [11]:
# replace all the '<error>' with '</error>' so that we can access it using xml.etree.ElementTree
#bg = bg.map(lambda line: re.sub(r'\s*<\s*error.*>\n|.*/error>\n', '', line))
# drop the content within the error tags <![CDATA[...]]>
#bg = bg.map(lambda line: re.sub(r'<\!.*', '', line))
# drop both error and content within the error tags <error ...><![CDATA[...]]></error>
bg = bg.map(lambda line: re.sub(r'\s*<\s*error.*>\n|<\!.*|.*/error>\n', '', line))

In [12]:
bg.take(5)

('<file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/pmd/mvn_apps/my-javacodeanalysis-app0/src/main/java/myjavacodeanalysis/Code_10000096_9914015_73_1.java">\n<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="ClassNamingConventions" ruleset="Code Style" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_codestyle.html#classnamingconventions" priority="1">\nThe class name \'Code_10000096_9914015_73_1\' doesn\'t match \'[A-Z][a-zA-Z0-9]*\'\n</violation>\n',
 '<viola

In [13]:
# test if the error tags are dropped
bg.filter(lambda line: line.find('<error') >= 0).take(5)

()

In [14]:
# test if the content drop the content within the error tags are dropped
bg.filter(lambda line: line.find('<![CDATA') >= 0).take(5)

()

### Drop wherever the file tage occurs
- Since we are intrested in getting the 'violations'
  

In [15]:
# replace all the '<file>' with '</file>' so that we can access it using xml.etree.ElementTree
#bg = bg.map(lambda line: re.sub(r'\s*<\s*file.*>|.*/file>', '', line))
bg = bg.map(lambda line: re.sub(r'\s*<\s*file.*>\n|.*/file>\n', '', line))

In [16]:
bg.take(5)

('<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="ClassNamingConventions" ruleset="Code Style" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_codestyle.html#classnamingconventions" priority="1">\nThe class name \'Code_10000096_9914015_73_1\' doesn\'t match \'[A-Z][a-zA-Z0-9]*\'\n</violation>\n',
 '<violation beginline="10" endline="10" begincolumn="23" endcolumn="31" rule="MethodArgumentCouldBeFinal" ruleset="Code Style" class="Code_10000096_9914015_73_1" method="setId" variable="id" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_jav

### Filter out just the violations

In [17]:
## Filter out just the rows
bg = bg.filter(lambda line: line.find('<violation') >= 0)
#bg = bg.filter(lambda line:  re.sub(r'\s*<\s*violation.*>\n|.*/violation>\n', '', line) >= 0)
#bg.filter(lambda line: line.find('<violation' or 'violation>') >= 0).take(5)
#bg.filter(lambda line:  re.findall(r'\s*<\s*violation.*>\n|.*/violation>\n', line)).take(5)
#bg = bg.filter(lambda line:  re.findall(r'\s*<\s*violation.*>\n.*|.*\n|<.*/violation>\n', line))

In [18]:
bg.take(5)

('<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="DataClass" ruleset="Design" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass" priority="3">\nThe class \'Code_10000096_9914015_73_1\' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n</violation>\n',
 '<violation beginline="2" endline="19" begincolumn="8" endcolumn="1" rule="ClassNamingConventions" ruleset="Code Style" class="Code_10000096_9914015_73_1" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_java_codestyle.html#classnamingconventions" priority="1">\nThe class name \'Code_10000096_9914015_73_1\' doesn\'t match \'[A-Z][a-zA-Z0-9]*\'\n</violation>\n',
 '<violation beginline="10" endline="10" begincolumn="23" endcolumn="31" rule="MethodArgumentCouldBeFinal" ruleset="Code Style" class="Code_10000096_9914015_73_1" method="setId" variable="id" externalInfoUrl="https://pmd.github.io/pmd-6.39.0/pmd_rules_jav

### Use ElementTree to get all the attrbutes and text in xmlor html tags

In [19]:
# This will give us just the attributes in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: et.fromstring(row).attrib)

In [20]:
# This will give us just the text enclosed in the violation tag: <violation ...>
#violations_rows_bd = violations_rows_bd.map(lambda row: {'text':et.fromstring(row).text})

- To get both the the attrbutes and text in the violation tag 
    - We use add_text_to_dict_attrb() to achieve that

In [21]:
# Add extra key value to dict if they do not exixt
def add_key_val_if_not_exist(_dict):
    _dict
    keys = [
        'beginline',
        'endline',
        'begincolumn',
        'endcolumn', 
        'rule',
        'ruleset',
        'class',
        'method',
        'externalInfoUrl',
        'priority',
        'msg'
    ]
    value=''
    for key in keys:
        if key not in _dict:
            _dict[key] = value
    return _dict

In [22]:
# Convert text in tag to key/value
def update_dict_key_wt_text_as_value(tag, value, _dict):
    if tag == 'violation': 
        _dict['msg'] = value
    else:
        _dict = add_key_val_if_not_exist({})
    return _dict

In [23]:
#https://stackoverflow.com/questions/43921237/access-nested-children-in-xml-file-parsed-with-elementtree
def content(root):
    _dict = {}
    #list_of_dicts = []
    try:
        for child in root.iter():
            # update dict with text as values
            _dict = update_dict_key_wt_text_as_value(child.tag, child.text, child.attrib)
            # update dict with keys that do not exist
            _dict = add_key_val_if_not_exist(_dict)
            #list_of_dicts.append(_dict)
    except ParseError:
        # pass an empty dict eo update all keys with empty values
        #list_of_dicts.append(add_key_val_if_not_exist({}))
        _dict = add_key_val_if_not_exist({})

    #return list_of_dicts
    return _dict

In [24]:
## Use ElementTree to get all the attrbutes in xml tags
#bg = bg.map(lambda row: et.fromstring(row).attrib)
bg = bg.map(lambda row: content(et.fromstring(row)))

In [25]:
bg.take(5)

({'beginline': '2',
  'endline': '19',
  'begincolumn': '8',
  'endcolumn': '1',
  'rule': 'DataClass',
  'ruleset': 'Design',
  'class': 'Code_10000096_9914015_73_1',
  'externalInfoUrl': 'https://pmd.github.io/pmd-6.39.0/pmd_rules_java_design.html#dataclass',
  'priority': '3',
  'msg': "\nThe class 'Code_10000096_9914015_73_1' is suspected to be a Data Class (WOC=0.000%, NOPA=0, NOAM=4, WMC=4)\n",
  'method': ''},
 {'beginline': '2',
  'endline': '19',
  'begincolumn': '8',
  'endcolumn': '1',
  'rule': 'ClassNamingConventions',
  'ruleset': 'Code Style',
  'class': 'Code_10000096_9914015_73_1',
  'externalInfoUrl': 'https://pmd.github.io/pmd-6.39.0/pmd_rules_java_codestyle.html#classnamingconventions',
  'priority': '1',
  'msg': "\nThe class name 'Code_10000096_9914015_73_1' doesn't match '[A-Z][a-zA-Z0-9]*'\n",
  'method': ''},
 {'beginline': '10',
  'endline': '10',
  'begincolumn': '23',
  'endcolumn': '31',
  'rule': 'MethodArgumentCouldBeFinal',
  'ruleset': 'Code Style',
  '

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [26]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-30:30",# walltime="0-00:30",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45878 instead
  warnings.warn(


In [27]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.140:39399 Dashboard: http://192.168.94.140:45878/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Flatten

In [28]:
#does not need to flatten
#bg = bg.flatten()

## Convert Bags to Dataframes

In [29]:
#convert from bags to dataframe
#ddf = bg.to_dataframe(meta={'name':'str', 'line':'int', 'severity':'str', 'message':'str', 'source':'str'})
ddf = bg.to_dataframe()

In [30]:
ddf

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
npartitions=1154,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


In [31]:
ddf.columns

Index(['beginline', 'endline', 'begincolumn', 'endcolumn', 'rule', 'ruleset',
       'class', 'externalInfoUrl', 'priority', 'msg', 'method'],
      dtype='object')

In [32]:
ddf.head()

,beginline,endline,begincolumn,endcolumn,rule,ruleset,class,externalInfoUrl,priority,msg,method
0,2,19,8,1,DataClass,Design,Code_10000096_9914015_73_1,https://pmd.github.io/pmd-6.39.0/pmd_rules_jav...,3,\nThe class 'Code_10000096_9914015_73_1' is su...,
1,2,19,8,1,ClassNamingConventions,Code Style,Code_10000096_9914015_73_1,https://pmd.github.io/pmd-6.39.0/pmd_rules_jav...,1,\nThe class name 'Code_10000096_9914015_73_1' ...,
2,10,10,23,31,MethodArgumentCouldBeFinal,Code Style,Code_10000096_9914015_73_1,https://pmd.github.io/pmd-6.39.0/pmd_rules_jav...,3,\nParameter 'id' is not assigned and could be ...,setId
3,16,16,25,35,MethodArgumentCouldBeFinal,Code Style,Code_10000096_9914015_73_1,https://pmd.github.io/pmd-6.39.0/pmd_rules_jav...,3,\nParameter 'name' is not assigned and could b...,setName
4,2,2,1,21,UnnecessaryImport,Code Style,Code_1001005_1000723_311_0,https://pmd.github.io/pmd-6.39.0/pmd_rules_jav...,4,\nUnused import 'com.sun.jna.*'\n,


In [33]:
#ddf.tail()

In [34]:
#ddf.shape[0].compute()
# 221495

### Make a folder in that directory

In [35]:
## Make a folder in that directory
folder = '{}/pmdcodesnippetsviolation_csv'.format(common_path)
# output: path/to/Post.csv => path/to
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

### Save files in that directory

In [36]:
## Save files in that directory
filename = 'pmdCodeSnippetsViolation'
file = '{}/{}*.csv'.format(folder, filename)
_ = ddf.to_csv(file, sep=',', index=False)

distributed.utils - ERROR - Timed out during handshake while connecting to tcp://10.65.4.149:42225 after 10 s
Traceback (most recent call last):
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/comm/tcp.py", line 196, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/asyncio/tasks.py", line 492, in wait_for
    fut.result()
asyncio.exceptions.CancelledError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/comm/core.py", line 318, in connect
    handshake = await asyncio.wait_for(comm.read(), time_left())
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl

KilledWorker: ("('bag-from-delayed-decode-filter-lambda-read-block-lambda-lambda-to_dataframe-3be0b6ad3c2f2cf32a8ba9bba69d9ad4', 550)", <Worker 'tcp://10.65.4.96:44991', name: 211-0, memory: 0, processing: 8>)